# Informe ECI 2019 - Introducción al Lenguaje Natural con Redes Neuronales

En este trabajo decidimos utilizar fastText para reproducir los resultados del paper. fastText es una libreria de clasificación y representación de texto. Lo realiza transformando el texto en vectores continuos.

Encontramos buenos resultados con los parametros default de fastText, pero realizaremos una busqueda de grilla (Grid Search) para encontrar los que den mejores resultados.

En esta notebook se pueden generar los archivos de train, dev y test, correr una validación contra dev y generar el archivo de predicciones de test en el formato de Kaggle.

In [6]:
import fasttext
import csv
import argparse
import json
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import math
sns.set()
plt.rcParams['figure.figsize'] = (10,7)
plt.rcParams['font.size'] = 23
sns.set_context('paper', font_scale=2)

### Funciones auxiliares para parsear texto

In [2]:
#!/usr/bin/env python
import argparse
import json
import csv

def generate_file(sentences_file, label_file, output):
    if label_file != None:
        for sentence, label in zip(it_sentences(sentences_file), it_labels(label_file)):
        	output.write("".join(["__label__",label," ",sentence," ","\n"]))
            # Tenemos la oración en sentence con su categoría en label
            #print(label, sentence)
    else:
        for sentence in it_sentences(sentences_file):
            output.write("%s\n" % sentence)
            # Tenemos una oración en sentence
            #print(sentence)
            pass
    

def it_sentences(sentence_data):
    for line in sentence_data:
        example = json.loads(line)
        yield example['sentence2']

def it_labels(label_data):
    label_data_reader = csv.DictReader(label_data)
    for example in label_data_reader:
        yield example['gold_label']

## Generamos los files de entrada

In [3]:
#Generate train data
generate_file(open("snli_1.0_train_filtered.jsonl", "r"), open("snli_1.0_train_gold_labels.csv", "r"), open("fast.txt", "w+"))
#Generate dev data
generate_file(open("snli_1.0_dev_filtered.jsonl", "r"), open("snli_1.0_dev_gold_labels.csv", "r"), open("fast_dev.txt", "w+"))
#Generate test data
generate_file(open("snli_1.0_test_filtered.jsonl", "r"), None, open("fast_test.txt", "w+"))

# Busqueda de parametros con Grid Search

En nuestro Grid Search vamos a considerar diferentes parametros que consideramos que pueden dar resultados interesantes: el learning rate, los epochs, el tamaño de la ventana de contexto de palabras, la función de perdida, el máximo tamaño de los ngrams y el tamaño del vector de palabras. Utilizando los parametros default de fastText conseguimos los siguientes resultados comparando contra el dataset de dev:

fastText usa como parametros default:
* learning rate: 0.1
* epoch: 5
* window: 5
* loss function: softmax
* nGram: 1
* dims: 100

In [5]:
def get_score(model):
    dev_data = open("fast_dev.txt", "r")
    result = 0
    total = 0
    for line in dev_data:
        split = line.split(" ", 1)
        result += model.predict(split[1].rstrip('\n'))[0][0][len("__label__"):] == split[0][len("__label__"):]
        total += 1
    return result*100/total

model = fasttext.train_supervised("fast.txt")
print("El porcentaje de aciertos es: %s" % get_score(model))

El porcentaje de aciertos es: 65.06807559439139


Con estos parametros no logramos replicar los resultados obtenidos en el paper. Esto nos motiva a realizar nuestro Grid Search sobre parametros que nos dieron buenos resultados en pruebas preliminares y que nos resultaron interesantes para experimentar.

Para correr el Grid Search (tarda aproximadamente 8 horas, adjuntamos el csv con los resultados):

In [ ]:
learning_rate = [0.01, 0.1, 0.25]
epochs = [5, 10, 100]
windows = [2, 5, 7]
loss = ["ova", "softmax"]
wordNgrams = [1, 2, 3]
dims = [100, 300]

for lr in learning_rate:
    for epoch in epochs:
        for ws in windows:
            for l in loss:
                for nGram in wordNgrams:
                    for dim in dims:
                        dev_data = open("fast_dev.txt", "r")
                        model = fasttext.train_supervised("fast.txt", lr = lr, epoch= epoch, ws= ws, loss=l, wordNgrams = nGram, dim=dim)
                        result = 0
                        total = 0
                        for line in dev_data:
                            split = line.split(" ", 1)
                            result += model.predict(split[1].rstrip('\n'))[0][0][len("__label__"):] == split[0][len("__label__"):]
                            total += 1
                        print("lr:%s, epoch:%s, window:%s, loss=%s, nGram=%s, dim:%s" % (lr, epoch, ws, l, nGram, dim))
                        print(result*100/total)
                        del model

Encontramos multiples combinaciones de parametros que logran tener una accuracy de 67% sobre el set de datos de validación, con ligeras variaciones entre si. Aun así no logramos ningun resultado superador al propuesto por el paper, por lo que podemos considerar que 67% es el mejor resultado para fastText. 

Algunas cosas que pudimos ver a partir de los resultados son:

* Si el learning rate va a ser bajo, necesita ser acompañado de muchos epochs para obtener buenos resultados. Esto parece dar buenos resultados de forma más consistente.
* No podamos dar ninguna afirmación sobre los otros parametros sobre los que experimentamos: conseguimos resultados aceptables con combinaciones de todos ellos, ten

Despues de correr contra el dataset de test conseguimos resultados similares, pudiendo replicar los resultados esperados.

### Generación de files de test en formato de Kaggle

Para conseguir un modelo bueno, nos vamos a quedar con el que mejor desempeño tenga contra el set de validación, utilizando los parametros que mejor resultados nos dieron en el experimento anterior. El que tenga mejores resultados sera enviado como nuestra respuesta.

In [4]:
model = fasttext.train_supervised("fast.txt", lr = 0.001, epoch= 100, ws= 2, loss="ova", wordNgrams = 3, dim=300)
best_score = get_score(model)
for i in range(0, 10):
    new_model = fasttext.train_supervised("fast.txt", lr = 0.001, epoch= 100, ws= 2, loss="ova", wordNgrams = 3, dim=300)
    score = get_score(new_model)
    if score > best_score:
        print(score)
        best_score = score
        del model
        model = new_model
    else:
        del new_model

67.39483844747002
67.40499898394636
67.41515952042268


In [14]:
def generate_answer():
    "Junta el archivo con las oraciones de test (jsonl)"
    " y los resultados de la clasificación de tu algoritmo (en tu formato)"
    " en un archivo csv compatible con el formato de Kaggle"

    sentences_filename = "snli_1.0_test_filtered.jsonl"
    labels_filename = "test_cls.txt"
    output_filename = "result.csv"

    with open(output_filename, 'w+') as fout:
        csv_writer = csv.writer(fout)
        csv_writer.writerow(['pairID', 'gold_label'])

        for pairID, label in it_ID_label_pairs(sentences_filename, labels_filename):
            formatted_label = label
            csv_writer.writerow([pairID, formatted_label])

def format_label(label):
    return label[len("__label__"):]

def it_ID_label_pairs(sentences_filename, labels_filename):
    sentence_data = open(sentences_filename, 'r')
    labels_data = open(labels_filename, 'r')
    for pairID, label in zip(it_ID(sentence_data), it_labels(labels_data)):
        yield pairID, label

def it_ID(sentence_data):
    for line in sentence_data:
        example = json.loads(line)
        yield example['pairID']

def it_labels(label_data):
    for label in label_data:
        label = label.rstrip('\n')  # sacamos el fin de linea
        yield label

In [15]:
#Genera el file de test
output = open("test_cls.txt", "w+")
test_data = open("fast_test.txt", "r")
for line in test_data:
    output.write("%s\n" % model.predict(line.rstrip('\n'))[0][0][len("__label__"):])
    
generate_answer()